# **More Data** -pt2

Import Modules:

In [1]:
import os
import pandas as pd
import json as js
import tmdbsimple as md


- Designate a folder to save your information:

In [15]:
OldFolder = 'Data'
Folder = "MoreData"

- Define the years you wish to retrieve:

In [16]:
Years = 'Number'

**Define any custom functions you will use:**

- Combine All the information we have extracted from the Last project to have it all available in one:

In [18]:
for i in os.listdir(OldFolder):
    print(i)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '\\Data'

In [12]:
# Combine The Data:
combined_df = combine_csv(OldFolder, Folder)
print(combined_df)

None


- **Create an OUTER loop for each year with a progress bar using tqdm_notebook:**